## 목차

1. 프로젝트 개요  
 1.1. 문제 정의 및 목표  
 1.2. 데이터셋 소개 및 문서 타입 선택  
 1.3. 요약 방식 및 평가 지표  

2. 환경 설정 및 데이터 준비  
 2.1. 환경 설정 및 라이브러리 버전 확인  
​ 2.2. 원본 JSON 데이터 로드 및 구조 파악  
​ 2.3. 전처리 및 Hugging Face Dataset 변환  ​
​
3. 요약 모델 구현 및 파인튜닝  
 3.1. [베이스 모델] KoBART 학습 설정  
 3.2. [대체 모델 1] KoT5 한국어 요약 모델  
 3.3. [대체 모델 2] mT5-small 한국어 요약 모델

4. 요약 생성, 평가 및 모델 비교  
 4.1. [모델 추론] 각 모델별 요약 생성  
​ 4.2. [정성 평가] 원문–정답–모델 요약 사례 비교  
 4.3. [정량 평가] ROUGE 계산 및 모델별 성능 비교 표  

5. 전체 워크플로우 정리 및 마무리  
 5.1. [파이프라인 정리] 모델별 학습 및 평가 요약  
 5.2. [실험 인사이트] 모델 비교 결과 정리 및 확장 아이디어 제안  

## 1. 프로젝트 개요

### 1.1. 문제 정의 및 목표

- 문제 정의
    - 이 프로젝트의 목표는 한국어 문서(신문 기사, 사설, 법률 문서)에 대해, 긴 원문을 짧고 핵심적인 요약문으로 자동 변환하는 추상적 문서 요약 모델을 구현·비교하는 것이다.
    - AI Hub 문서요약 데이터셋을 기반으로, 세 가지 Transformer 계열 모델을 동일한 설정으로 파인튜닝하고, ROUGE 등 지표를 통해 성능을 비교·분석한다.

- 프로젝트 목표
    - 한국어 문서요약 데이터의 구조를 이해하고, 요약 모델 입력에 적합한 형태로 가공하는 전체 파이프라인을 구축한다.
    - KoBART, KoT5, mT5-small 세 모델을 동일한 데이터와 에포크 조건에서 학습시켜, 모델 구조·크기에 따른 요약 성능 차이를 분석한다.
    - 정성적 사례(원문–정답–모델 요약)와 정량적 지표(ROUGE)를 함께 사용해, 어떤 모델이 어떤 상황에서 더 적합한지 논리적으로 해석한다.

### 1.2. 데이터셋 소개 및 문서 타입 선택

- 데이터셋 소개
    - AI Hub에서 제공하는 한국어 문서요약 데이터셋을 사용한다.
    
    - 데이터는 JSON 형식으로 제공되며, 각 문서는 documents 리스트 안에 text(문장 단위 원문)와 abstractive(사람이 작성한 요약문) 필드로 구성되어 있다.
​
    - 전체 데이터셋은 신문 기사, 사설, 법률 등 여러 문서 타입을 포함하지만, 본 실험에서는 베이스라인 코드와 동일하게 사설(editorial) 도메인만 선택하여 학습 및 평가를 진행한다.  
    \*train_original_editorial.json과 valid_original_editorial.json 파일을 사용하여 학습용·검증용 데이터셋 구성

### 1.3. 요약 방식 및 평가 지표

- 요약 방식
    - 본 프로젝트에서 다루는 요약 방식은 추상적(abstractive) 단일 문서 요약이다.
    - 각 입력은 하나의 긴 기사(사설)이며, 출력은 사람이 작성한 abstractive 요약과 유사한 형식의 자연어 요약문이다.
    - KoBART, KoT5, mT5-small 모두 시퀀스-투-시퀀스(Seq2Seq) Transformer 구조를 사용하여 원문 시퀀스를 요약 시퀀스로 직접 생성한다.

- 평가 지표
    - 정성적·정량적 방법을 함께 사용한다.
    - 정성적 평가는 여러 예시에서 원문–정답 요약–모델 요약을 나란히 비교하면서, 핵심 정보 보존, 논리 구조 유지, 문장 자연스러움 등을 관찰하는 방식으로 진행한다.
    - 정량적 평가지표로는 요약 분야에서 널리 쓰이는 `ROUGE-1`, `ROUGE-2`, `ROUGE-L`을 활용해, 생성된 요약과 정답 요약 사이의 n-그램 중복과 최장 공통 부분 수열을 측정한다. 

## 2. 환경 설정 및 데이터 준비

### 2.1. 환경 설정 및 라이브러리 버전 확인

- CUDA 활성화된 로컬 WSL(Linux) 환경에서 진행
- 파이썬 실행 환경과 핵심 라이브러리 버전 확인

In [1]:
import sys
print(sys.executable)

import torch, transformers, accelerate, peft
print(torch.__version__)
print(transformers.__version__)
print(accelerate.__version__)
print(peft.__version__)

/home/kmw/github/mission12/.venv/bin/python
2.7.1+cu118
4.57.3
1.12.0
0.17.1


### 2.2. 원본 JSON 데이터 로드 및 구조 파악

- load_json_dataset(file_path) 함수를 정의해 documents 리스트를 순회하면서, 각 문서의 문장 리스트(text 안의 이중 리스트 구조에서 sentence 필드)를 하나의 긴 문자열 full_text로 이어 붙인다.
​
- 각 문서에 대해 요약은 abstractive 리스트의 첫 번째 요소(abstractive[0])를 사용해 summary로 저장하며, 최종적으로 {"text": full_text, "summary": summary} 형태의 딕셔너리 리스트를 반환한다.  
    \*해당 리스트가 이후 Hugging Face Dataset.from_list로 바로 변환될 수 있도록 설계한다.

In [2]:
import os
import json

# 기본 경로 설정
base_path = "/home/kmw/github/mission12/dataset/summarization"

# 관련 파일 하나만 불러오기
train_file = "train_original_editorial.json"
valid_file = "valid_original_editorial.json"

In [3]:
# 첫 문서 구조 확인
sample_path = os.path.join(base_path, train_file)
with open(sample_path, "r", encoding="utf-8") as f:
    sample = json.load(f)

first_doc = sample["documents"][0]
print(first_doc.keys())
print(first_doc["text"][:1])       # text의 첫 sublist
print(first_doc["abstractive"])    # 요약 리스트

dict_keys(['id', 'category', 'media_type', 'media_sub_type', 'media_name', 'size', 'char_count', 'publish_date', 'title', 'text', 'annotator_id', 'document_quality_scores', 'extractive', 'abstractive'])
[[{'index': 0, 'sentence': '이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장서야 한다. 이미 상당한 변화의 조짐이 있다는 것을 고맙게 생각한다. 총수들께서 직접 관심을 가져주시면 빨리 전파돼 긍정적인 평가를 받을 수 있다고 본다"고 말했다.', 'highlight_indices': '9,11;37,39;53,55;91,93;104,106'}, {'index': 1, 'sentence': "언뜻 보아 무슨 말인지 불분명하나 이 대통령이 지난 8ㆍ15 연설 후 정몽준 의원, 정몽구 현대차 회장이 각각 2000억원과 5000억원을 기부한 사실과 '공생발전'이란 화두를 연결하면 금방 짐작이 간다.", 'highlight_indices': '0,2;6,8;14,16;59,61;104,106'}, {'index': 2, 'sentence': '다른 그룹 총수들도 좀 나서라고 은근히 떠민 것이다.', 'highlight_indices': '0,2;11,12;18,21'}, {'index': 3, 'sentence': '이 대통령은 기부에 대한 후속 선언이 나오지 않은 탓인지 총수들의 사회공헌 방안에 불만을 표시했다는 후문이다.', 'highlight_indices': '0,1'}]]
['이명박 대통령은 어제 30대 그룹 총수를 모아놓고 시대적 요구는 역시 총수가 앞장서야 한다고 발언하며 기부문화 확산을 은근히 강조했으나 대통령의 강요나 포퓰리즘에 의한 압박보다는 자발적 문화로 구축해야 효과가 더 큰 법으로 방안 마련을 맡기고 시간을 줘야 할 것으로 보인다.']

In [4]:
def load_json_dataset(file_path):
    # 하나의 JSON 파일을 열어 문서별로 text, summary 정보를 추출
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    examples = []
    for doc in data["documents"]:
        sentences = []
        # "text"는 중첩 리스트 형태이므로 내부의 모든 sentence를 추출
        for sublist in doc["text"]:
            for item in sublist:
                sentences.append(item.get("sentence", ""))
        full_text = " ".join(sentences)
        # abstractive 요약은 첫번째 항목 사용 (없으면 빈 문자열)
        summary = doc["abstractive"][0] if doc["abstractive"] else ""
        examples.append({
            "text": full_text,
            "summary": summary,
        })
    return examples

In [5]:
# OOM 발생으로 샘플 수 축소
train_examples = load_json_dataset(os.path.join(base_path, train_file))
train_examples = train_examples[:5000]
valid_examples = load_json_dataset(os.path.join(base_path, valid_file))
valid_examples = valid_examples[:1000]

print("Train examples:", len(train_examples))
print("Validation examples:", len(valid_examples))

Train examples: 5000
Validation examples: 1000


### 2.3. 전처리 및 Hugging Face Dataset 변환
- `text–summary` 쌍 리스트를 `Hugging Face Dataset/DatasetDict`로 변환한다.

- `gogamza/kobart-base-v1` 토크나이저를 불러와 `KoBART`에 맞는 입력·레이블 시퀀스를 생성한다.

- `tokenize_function`에서 원문은 최대 512 토큰, 요약은 최대 128 토큰으로 잘라낸다.

- `.map()`을 사용해 이 토큰화 과정을 train/validation 전체에 일괄 적용한다.

- 준비된 `tokenized_datasets`를 바로 `Trainer`에 입력하여 파인튜닝에 사용한다.

In [6]:
from datasets import Dataset, DatasetDict
from transformers import BartTokenizer

# train_examples와 valid_examples로 Dataset 객체 생성
train_dataset = Dataset.from_list(train_examples)
valid_dataset = Dataset.from_list(valid_examples)

# DatasetDict 형태로 통합 (훈련/검증)
dataset = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset
})

In [7]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import BartTokenizerFast, DataCollatorForSeq2Seq

model_name = "gogamza/kobart-base-v1"
tokenizer = BartTokenizerFast.from_pretrained(model_name)

def tokenize_function(example):
    model_inputs = tokenizer(
        example["text"],
        max_length=512,
        truncation=True
    )
    labels = tokenizer(
        text_target=example["summary"],
        max_length=128,
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## 3. 요약 모델 구현 및 파인튜닝

### 3.1. [베이스 모델] KoBART 학습 설정

- 모델: gogamza/kobart-base-v1 (한국어 BART 기반 Seq2Seq 요약 모델)

- 입력 데이터: train_original_editorial.json / valid_original_editorial.json에서 추출한 사설 도메인 text–summary 쌍

- 토큰화:
    - 최대 입력 길이: 512 토큰
    - 최대 요약 길이: 128 토큰
    - BartTokenizerFast 사용

- 학습 설정:
    - 에포크 수: 3(기본 설정)
    - Trainer 사용 (train: tokenized_datasets["train"], eval: tokenized_datasets["validation"])
    - 데이터 콜레이터: DataCollatorForSeq2Seq

In [8]:
from transformers import BartForConditionalGeneration, TrainingArguments, Trainer

checkpoint = "gogamza/kobart-base-v1"
model = BartForConditionalGeneration.from_pretrained(checkpoint)
training_args = TrainingArguments(output_dir="test-trainer", report_to="none")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
/tmp/ipykernel_4631/1462772139.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,2.196200
1000,1.707800
1500,1.455400


/home/kmw/github/mission12/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1875, training_loss=1.6923163736979168, metrics={'train_runtime': 869.9496, 'train_samples_per_second': 17.242, 'train_steps_per_second': 2.155, 'total_flos': 4300009174056960.0, 'train_loss': 1.6923163736979168, 'epoch': 3.0})

### 3.2. [대체 모델 1] KoT5 한국어 요약 모델

- 모델: psyche/KoT5-summarization (한국어 T5 기반 요약 사전학습 모델)  
    \*최초 대체 모델 1은 `T5-base`로 선정하였으나, OOM 발생으로 해당 모델로 변경하여 실험을 진행하였다.

- 입력 데이터 / 전처리:
    - 2장에서 정의한 train_examples / valid_examples를 그대로 사용
    - 동일한 text–summary 쌍, 최대 길이는 GPU 메모리를 고려해 입력 512, 요약 128 토큰으로 설정

- 토크나이저 및 토큰화:
    - AutoTokenizer를 사용해 psyche/KoT5-summarization 전용 토크나이저를 로드
    - 원문(text)을 인코더 입력으로, 요약문(summary)을 디코더 라벨로 사용하는 Seq2Seq 방식 적용

- 학습 설정:
    - 에포크 수: 3 (KoBART와 통일)
    - 배치 크기, 학습률 등은 KoBART와 최대한 비슷하게 맞춰 공정한 비교를 목표로 함

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

kot5_model_name = "psyche/KoT5-summarization"

# KoT5 토크나이저 및 모델 로드
kot5_tokenizer = AutoTokenizer.from_pretrained(kot5_model_name)
kot5_model = AutoModelForSeq2SeqLM.from_pretrained(kot5_model_name)

# KoT5 토큰화 함수 (입력 512, 요약 128 유지)
def kot5_tokenize_function(example):
    model_inputs = kot5_tokenizer(
        example["text"],
        max_length=512,
        truncation=True,
    )
    labels = kot5_tokenizer(
        text_target=example["summary"],
        max_length=128,
        truncation=True,
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 원본 dataset에서 KoT5용 토큰화
kot5_tokenized_datasets = dataset.map(kot5_tokenize_function, batched=True)

# KoT5용 DataCollator
kot5_data_collator = DataCollatorForSeq2Seq(
    tokenizer=kot5_tokenizer,
    model=kot5_model,
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
from transformers import TrainingArguments, Trainer

kot5_training_args = TrainingArguments(
    output_dir="kot5-summarization",
    num_train_epochs=3,
    report_to="none",
    # 필요하다면 여기서 per_device_train_batch_size, learning_rate 등을
    # KoBART, mT5-small과 맞춰서 추가 설정
)

kot5_trainer = Trainer(
    model=kot5_model,
    args=kot5_training_args,
    train_dataset=kot5_tokenized_datasets["train"],
    eval_dataset=kot5_tokenized_datasets["validation"],
    data_collator=kot5_data_collator,
    tokenizer=kot5_tokenizer,
)

kot5_trainer.train()

/tmp/ipykernel_3457/2489240065.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  kot5_trainer = Trainer(


Step,Training Loss
500,1.691500
1000,1.609700
1500,1.542900


TrainOutput(global_step=1875, training_loss=1.5954128580729168, metrics={'train_runtime': 30904.5178, 'train_samples_per_second': 0.485, 'train_steps_per_second': 0.061, 'total_flos': 8578932157440000.0, 'train_loss': 1.5954128580729168, 'epoch': 3.0})

### 3.3. [대체 모델 2] mT5-small 한국어 요약 모델

- 모델: google/mt5-small

- 입력 데이터 / 전처리:
    - 2장에서 준비한 dataset (사설 도메인 text–summary 쌍) 그대로 사용
    - 최대 길이: 입력 512 토큰, 요약 128 토큰 (KoBART, KoT5와 동일)

- 특징:
    - 모델 크기가 작아, 같은 3 epoch 기준으로 GPU 메모리와 학습 시간이 상대적으로 적게 든다.
    - 한국어 전용은 아니므로, 한국어 특화 모델(KoBART, 한국어 KoT5)과의 성능 차이를 비교하기 좋다.

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

mt5_model_name = "google/mt5-small"

# mT5-small 토크나이저 및 모델 로드
mt5_tokenizer = AutoTokenizer.from_pretrained(mt5_model_name)
mt5_model = AutoModelForSeq2SeqLM.from_pretrained(mt5_model_name)

# mT5-small용 토큰화 함수 (KoBART/T5와 동일한 max_length 사용)
def mt5_tokenize_function(example):
    model_inputs = mt5_tokenizer(
        example["text"],
        max_length=512,
        truncation=True
    )
    labels = mt5_tokenizer(
        text_target=example["summary"],
        max_length=256,
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 동일한 원본 dataset에서 mT5-small용 토큰화
mt5_tokenized_datasets = dataset.map(mt5_tokenize_function, batched=True)

# mT5-small용 DataCollator
mt5_data_collator = DataCollatorForSeq2Seq(tokenizer=mt5_tokenizer, model=mt5_model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/kmw/github/mission12/.venv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments, Trainer

# mT5-small 학습 설정
mt5_training_args = TrainingArguments(
    output_dir="mt5-small-summarization",
    num_train_epochs=3,
    report_to="none",
)

mt5_trainer = Trainer(
    model=mt5_model,
    args=mt5_training_args,
    train_dataset=mt5_tokenized_datasets["train"],
    eval_dataset=mt5_tokenized_datasets["validation"],
    data_collator=mt5_data_collator,
    tokenizer=mt5_tokenizer,
)

# mT5-small 학습
mt5_trainer.train()

/tmp/ipykernel_395/3429381621.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  mt5_trainer = Trainer(


Step,Training Loss
500,5.160900
1000,3.070100
1500,2.909500


TrainOutput(global_step=1875, training_loss=3.5427413411458333, metrics={'train_runtime': 11168.9935, 'train_samples_per_second': 1.343, 'train_steps_per_second': 0.168, 'total_flos': 7931245363200000.0, 'train_loss': 3.5427413411458333, 'epoch': 3.0})

# 파인튜닝된 모델로 pipeline 추론 테스트

## 4. 요약 생성, 평가 및 모델 비교

### 4.1. [모델 추론] 각 모델별 요약 생성

**[베이스 모델] KoBART**

In [9]:
from transformers import pipeline

# summarization pipeline 생성 (모델과 토크나이저가 이미 정의되어 있다고 가정)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# 검증 데이터셋의 첫 10개 예시에 대해 요약 결과 출력
for i in range(10):
    sample_text = tokenized_datasets["validation"][i]["text"]
    prompt = f"원문:\n{sample_text}\n요약:"
    print(f"예시 {i+1}:")
    print(prompt)

    # 요약 결과 생성 (필요한 max/min length는 상황에 맞게 조정)
    result = summarizer(prompt, max_length=130, min_length=30, do_sample=False)
    print(result[0]['summary_text'])
    print("-" * 50)

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


예시 1:
원문:
더불어민주당 이해찬 대표가 30 일 오후 국회에서 기자간담회를 열고 조국 전 법무부 장관 사태와 관련해 "국민 여러분께 매우 송구하다"고 밝혔다. 더불어민주당 이해찬 대표가 30 일 기자간담회를 열고 '조국 사태'와 관련, "국민 여러분께 매우 송구하다"는 입장을 밝혔다. 이 대표는 "검찰 개혁이란 대의에 집중하다 보니, 국민 특히 청년이 느꼈을 불공정에 대한 상대적 박탈감, 좌절감을 깊이 있게 헤아리지 못했다"며 "여당 대표로서 무거운 책임감을 느낀다"고 머리를 숙였다. 조국 전 법무부 장관이 14 일 사퇴한 이후 이 대표가 당 안팎의 쇄신 요구에 대해 입장을 표명한 것은 이번이 처음이다. 청와대와 여당은 '조국 정국'을 거치며 분출된 '공정'과 '정의'의 민심을 받들어 검찰 개혁에 매진하겠다면서도 두 달간 극심한 분열과 갈등을 초래한데 대해선 진지하게 성찰하는 모습을 보이지 않았다. 그나마 초선인 이철희 의원이 "당이 대통령 뒤에 비겁하게 숨어 있었다"고 비판했고, 표창원 의원은 "책임을 느끼는 분들이 각자 형태로 그 책임감을 행동으로 옮겨야 할 때"라고 지적했다. 뒤늦게나마 이 대표가 자성의 목소리를 내긴 했으나 당 안팎의 쇄신 요구에 어떻게 응할지 구체적 플랜을 제시하지 못해 여전히 안이하다는 지적도 나온다. 이 대표는 28 일 윤호중 사무총장을 단장으로 하는 총선기획단을 발족했고 조만간 인재영입위원회도 출범시킬 계획이라고 밝혔다. 이 대표는 "민주당의 가치를 공유하는 참신한 인물을 영입해 준비된 정책과 인물로 승부하겠다"고 다짐했다. 하지만 당 일각에선 "총선기획단장을 비롯한 당직 인선부터 쇄신 의지를 보여야 한다"는 비판의 목소리가 나온다. 무조건 물러나는 게 능사는 아니지만 국정 혼선을 초래한 데 대해 당 지도부가 겸허하게 책임지는 모습을 보이는 게 쇄신의 출발점이 돼야 한다는 지적도 있다. 선거는 대중의 이해와 요구를 잘 대표하는 정치인을 뽑는 행위다. 민생을 외면하며 낡은 이념과 진영 싸움에 매몰된 구시대 인물들을 과감히 물갈

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


민주당 이해찬 대표가 30 일 기자간담회를 열고 조국 전 법무부 장관 사태와 관련해 "국민 여러분께 송구하다"는 입장을 밝혔으나 당 안팎의 쇄신 요구에 대한 구체적인 플랜을 제시하지 못해 안이하다는 지적도 나오고 있다.  4 차 산업혁명의 거센 파고를 헤쳐나갈 전문성을 갖춘 젊고 유능한 인재들을 널리 구해야 할 것이다.     이해찬 대표의 유감 표명이 여권 전반의 인적 쇄신으로 이어지길 기대한다.      인적 쇄신으로 이어지길 기대한다.                 인적 쇄신으로 이어지길 기대한다.                            민생을 외면한 구시대 인물들을 과감히 물갈이하라는 국민의 요구도 국민의 요구다.                                     인적 쇄신으로 이어지길 바란다.           인적 쇄신으로 이어지길 기대한다.                 
--------------------------------------------------
예시 2:
원문:
탈원전 정책에서 비롯된 한국전력의 경영 부담이 결국 전기료 인상을 초래하게 됐다. 김종갑 한전 사장은 언론 인터뷰에서 "정부 정책에 따라 도입된 각종 전기료 특례할인을 모두 폐지하고 전기요금 원가공개 방안을 정부와 협의하겠다"고 밝혔다. 특례할인은 여름철 누진제, 주택용 절전 및 전기차 충전 등의 명목으로 전기료를 깎아주는 제도로서, 작년 1 조 1434 억원에 달했다. 이 제도가 없어지면 부담은 고스란히 국민에게 돌아가게 된다. 전기료 인상은 예견됐던 것이나 마찬가지다. 문재인 정부 출범 이후 탈원전 정책이 가속화되면서 한전 경영이 악화일로를 달려왔기 때문이다. 원전 가동을 줄이고 발전 단가가 높은 민간 발전사의 전기 구매를 늘린 탓에 한전의 발전 원가는 2016 년 25 조원 수준에서 2018 년 38 조원 수준으로 급등했다. 영업이익이 같은 기간 12 조 16 억원 흑자에서 2080 억원 적자로 급반전한 것은 필연적인 결과다. 적자는 올해 상반기 

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

원문:부산경남(PK) 정치권이 대구경북 통합신공항 이전에 트집을 잡고 있는데, 이는 김해신공항을 무산시키고 가덕도 신공항을 추진하려는 부울경 정치권과 현 정권의 복심을 대변하는 것이다.   대구경북 통합신공항 선정 기준을 두고 대구시와 경북도가 최종 절충 방안을 제시한 가운데 연내 이전지 확정이라는 배수진을 치고 있는 상황인데 군위군수에 대한 경찰의 압수수색과 피의자 신분 조사를 둘러싸고 통합신공항 이전 추진에 악재로 작용하지 않을까 우려하는 목소리도 많다.                           부산경남 정치권의 신공항 딴죽걸기는 어제만일 아니다.        부산경남 정치권의 신공항 딴죽걸기는 어제오늘의 일이 아니다.         부산경남 정치권의 신공항 딴죽걸기는 어제만일 것이다.       부산경남 정치권의 신공항 딴죽걸기는 어제만일 것이다.           이들의 저의는 뻔하다.   김해신공항을 무산시키고 가덕도 신공항을 추진하려는 부울경 정치권과 현 정권의 복심을 대변하는 것이다.        
--------------------------------------------------
예시 8:
원문:
이주노동자들에게 적용되는 '출국 후 퇴직금 수령제도'로 인해 이주노동자들이 퇴직금을 받는데 어려움을 겪고 있다. 불법체류를 막기 위해 이주노동자가 출국한 뒤 퇴직금을 받도록 한 '출국만기보험 제도'가 퇴직금 미지급자를 양산하고 있는 것이다. 출국만기보험이란 사업주가 퇴직금을 지급하기 위해 노동자를 피보험자로 해 가입하는 보험이다. 사업주가 매달 통상임금의 8.3%를 보험회사에 적립한 것으로, 노동자가 퇴직 후 출국할 때 공항에서 받을 수 있다. 2014 년 정부는 불법체류 방지를 위해 이주노동자 퇴직금을 '퇴사 후 14 일 이내'에서 '출국 후 14 일 이내' 지급하는 것으로 바꿨다. 하지만 상당수 이주노동자들이 퇴직금 받는 방법을 제대로 모르고 있다. 이주노동자가 퇴직금을 받으려면 구비서류가 많은데다 퇴직금을 수령받는 절차도 복잡하다. 최

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


불법체류 방지를 위해 이주노동자가 출국한 뒤 퇴직금을 받도록 한 '출국만기보험 제도'가 퇴직금 미지급자를 양산하고 있는 가운데, 이주노동자에게만 그 시점을 다르게 적용하는 것은 기본권 침해라는 지적이 나오고 있다.   이주노동자에게만 그 시점을 다르게 적용하는 것은 기본권 침해라는 지적이 나오고 있다.   이주노동자에게만 그 시점을 다르게 적용하는 것은 기본권 침해라는 것이다.   이주노동자에게만 그 시점을 다르게 적용하는 것은 기본권 침해라는 지적도 나오고 있다.   이주노동자에게만 그 시점을 다르게 적용하는 것은 기본권 침해라는 지적도 나오고 있다.   이주노동자에게만 그 시점을 다르게 적용하는 것은 기본권 침해라는 지적도 나오고 있다.   이주노동자에게만 그 시점을 다르게 적용하는 것은 기본권 침해라는 지적도 나오고 있다.  이주노동자에게만 그 시점을 다르게 적용하는 것은 기본권 침해라는 지적도 나오고 있다.  이주노동자에게만 그 시점을 다르게 적용하는 것은 기본권 침해라는 지적도 나오고 있다.   이주노동자에게만 그 시점을 '출국'으로 다르게 적용하는 것은 기본권 침해라는 지적도 나오고 있다.   이주노동자에게만 그 시점을 다르게 적용하는 것은 기본권 침해라는 지적도 나오고 있다.  이주노동
--------------------------------------------------
예시 9:
원문:
법무부가 제정한 훈령 '형사사건 공개 금지 등에 관한 규정'을 둘러싼 논란이 뜨겁다. 당장 국민의 알권리와 정면 충돌하고 있다는 지적이 비등하고 있다. 법무부의 훈령은 사건 관계인의 인권보호를 위해 검찰의 수사 상황을 전면 비공개하고, 오보 언론에 대한 처벌 수단을 마련하겠다는 게 그 요지다. 구체적으로는 검사와 수사관 등 관계자가 기자와 개별적으로 만날 수 없도록 했다. 또 검찰수사와 관련해 오보를 내면 해당 언론사는 브리핑은 물론 검찰청 출입 자체가 제한된다. 다만 공보 담당자와는 만날 수 있다. 한마디로 기자는 알려주는 내용만 받아쓰라는 것이다. 언론 

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


법무부가  '형사사건 공개 금지 등에 관한 규정'을 둘러싼 논란이 뜨겁고 있는데 이는 지나친 피의사실 공개로 피의자 인권이 침해당하는 폐해를 막기 위한 조치라는게 법무부 설명인데 검찰이 망신을 주고, 사실상 여론재판이 이뤄지는 바람에 공정한 재판을 받을 피의자의 권리가 심각하게 훼손되는 경우가 다반사인데 정부가 특정 사건을 은폐 축소하더라도 이를 견제할 방법도 없다.  정부가 특정 사건을 은폐 축소하더라도 이를 견제할 방법도 없다.  언론 자유의 침해가 심각하게 우려되는 조치이며 정부가 특정 사건을 은폐 축소하더라도 이를 견제할 방법도 없다.  오히려 과거 적폐가 되살아날 수도 있다.  정부가 특정 사건을 은폐 축소하더라도 이를 견제할 방법도 없다.  정부는 특정 사건을 은폐 축소하더라도 이를 견제할 방법도 없다.  오히려 과거 적폐가 되살아날 수도 있다.   정부가 특정 사건을 은폐 축소해도 이를 견제할 방법도 없다.   정부는 규정 시행을 밀어붙이면 언론 종사사들 간에 자율적으로 판단하도록 해야 한다.    정부는 규정 시행을 밀어붙이면 언론 종사사들 스스로 알아서 판단하도록 해야 한다.       정부가 규정 시행을 밀어붙인다면 비판 언론에 재잘거리게 될 것이다.      정부는 규정 시행을 강행한다면 비판 언론에 재잘거리게 될 것이다. 
--------------------------------------------------
예시 10:
원문:
딱 한달이다. 두달도 이어가지 못했다. 5 개월만에 생산·소비·투자 등 경제 활동 3 대 지표가 모두 상승하며 희망을 줬던 산업활동동향(8 월)이 다시 한숨 속 위기감으로 빠져드는데는 한달이면 충분했다. 통계청이 31 일 발표한 '9 월 산업활동 동향'에서 지난달 전(全)산업 생산지수(계절조정계열)는 108.0 으로, 전월보다 0.4% 감소했다. 광공업 생산은 증가했지만, 도소매와 금융·보험업을 중심으로 서비스업 생산이 줄어든 것이 영향을 미쳤다. 소매판매는 음식료품과 의복 판매가 감소하면서 전월보다 2.2%줄

**[대체 모델1] KoT5 한국어 요약 모델**

In [12]:
from transformers import pipeline

# KoT5 summarization pipeline 생성
kot5_summarizer = pipeline(
    "summarization",
    model=kot5_model,        # 학습된 KoT5 모델
    tokenizer=kot5_tokenizer,
    device=0                 # GPU, CPU면 -1
)

# 검증 데이터셋의 첫 10개 예시에 대해 요약 결과 출력
for i in range(10):
    sample_text = kot5_tokenized_datasets["validation"][i]["text"]
    prompt = f"원문:\n{sample_text}\n요약:"
    print(f"예시 {i+1}:")
    print(prompt)

    result = kot5_summarizer(
        prompt,
        max_length=130,
        min_length=30,
        do_sample=False
    )
    print(result[0]["summary_text"])
    print("-" * 50)

Device set to use cuda:0


예시 1:
원문:
더불어민주당 이해찬 대표가 30 일 오후 국회에서 기자간담회를 열고 조국 전 법무부 장관 사태와 관련해 "국민 여러분께 매우 송구하다"고 밝혔다. 더불어민주당 이해찬 대표가 30 일 기자간담회를 열고 '조국 사태'와 관련, "국민 여러분께 매우 송구하다"는 입장을 밝혔다. 이 대표는 "검찰 개혁이란 대의에 집중하다 보니, 국민 특히 청년이 느꼈을 불공정에 대한 상대적 박탈감, 좌절감을 깊이 있게 헤아리지 못했다"며 "여당 대표로서 무거운 책임감을 느낀다"고 머리를 숙였다. 조국 전 법무부 장관이 14 일 사퇴한 이후 이 대표가 당 안팎의 쇄신 요구에 대해 입장을 표명한 것은 이번이 처음이다. 청와대와 여당은 '조국 정국'을 거치며 분출된 '공정'과 '정의'의 민심을 받들어 검찰 개혁에 매진하겠다면서도 두 달간 극심한 분열과 갈등을 초래한데 대해선 진지하게 성찰하는 모습을 보이지 않았다. 그나마 초선인 이철희 의원이 "당이 대통령 뒤에 비겁하게 숨어 있었다"고 비판했고, 표창원 의원은 "책임을 느끼는 분들이 각자 형태로 그 책임감을 행동으로 옮겨야 할 때"라고 지적했다. 뒤늦게나마 이 대표가 자성의 목소리를 내긴 했으나 당 안팎의 쇄신 요구에 어떻게 응할지 구체적 플랜을 제시하지 못해 여전히 안이하다는 지적도 나온다. 이 대표는 28 일 윤호중 사무총장을 단장으로 하는 총선기획단을 발족했고 조만간 인재영입위원회도 출범시킬 계획이라고 밝혔다. 이 대표는 "민주당의 가치를 공유하는 참신한 인물을 영입해 준비된 정책과 인물로 승부하겠다"고 다짐했다. 하지만 당 일각에선 "총선기획단장을 비롯한 당직 인선부터 쇄신 의지를 보여야 한다"는 비판의 목소리가 나온다. 무조건 물러나는 게 능사는 아니지만 국정 혼선을 초래한 데 대해 당 지도부가 겸허하게 책임지는 모습을 보이는 게 쇄신의 출발점이 돼야 한다는 지적도 있다. 선거는 대중의 이해와 요구를 잘 대표하는 정치인을 뽑는 행위다. 민생을 외면하며 낡은 이념과 진영 싸움에 매몰된 구시대 인물들을 과감히 물갈

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


더불어민주당 이해찬 대표가 30일 기자간담회를 열고 조국 전 법무부 장관 사태와 관련해 "국민 여러분께 매우 송구하다"는 입장을 밝혔으나 당 안팎의 쇄신 요구에 어떻게 응할지 구체적 플랜을 제시하지 못해 여전히 안이하다는 지적도 나오며 이날 유감 표명이 여권 전반의 대대적인 인적 쇄신으로 이어지길 기대한다.
--------------------------------------------------
예시 2:
원문:
탈원전 정책에서 비롯된 한국전력의 경영 부담이 결국 전기료 인상을 초래하게 됐다. 김종갑 한전 사장은 언론 인터뷰에서 "정부 정책에 따라 도입된 각종 전기료 특례할인을 모두 폐지하고 전기요금 원가공개 방안을 정부와 협의하겠다"고 밝혔다. 특례할인은 여름철 누진제, 주택용 절전 및 전기차 충전 등의 명목으로 전기료를 깎아주는 제도로서, 작년 1 조 1434 억원에 달했다. 이 제도가 없어지면 부담은 고스란히 국민에게 돌아가게 된다. 전기료 인상은 예견됐던 것이나 마찬가지다. 문재인 정부 출범 이후 탈원전 정책이 가속화되면서 한전 경영이 악화일로를 달려왔기 때문이다. 원전 가동을 줄이고 발전 단가가 높은 민간 발전사의 전기 구매를 늘린 탓에 한전의 발전 원가는 2016 년 25 조원 수준에서 2018 년 38 조원 수준으로 급등했다. 영업이익이 같은 기간 12 조 16 억원 흑자에서 2080 억원 적자로 급반전한 것은 필연적인 결과다. 적자는 올해 상반기 9285 억원으로 더 불어났다. 우량회사가 2 년여 만에 적자의 수렁에 처박힌 꼴이다. 한전의 추락은 국제신용평가사들의 평가에도 그대로 적용된다. S&P는 그제 한전의 신용등급을 기존 BBB에서 BBB-로 내렸다. 탈원전 정책의 충격이 커지면서 재무 부담이 가중되고 있다는 게 하향 조정 이유다. 표면적으로는 '재무 부담'이란 용어를 사용했지만 한전의 기초체력이 급속히 악화되고 있다는 경고와 다를 바 없다. S&P가 한전의 주요 재무지표에 대해 앞으로 1~2 년간 개선되기 어렵다고 밝힌 점으로 미뤄 신용

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

김해가 지역구인 더불어민주당 김정호 국회의원이 28일 국회 예산결산특별위원회 전체회의에서 대구공항 통합이전과 관련한 타당성 용역보고서 제출을 요구하는 등 부산경남 정치권이 대구경북 통합신공항 이전에 트집을 잡고 나섰는데 이럴수록 지역 민심과 여론을 한곳으로 모아야 한다.
--------------------------------------------------
예시 8:
원문:
이주노동자들에게 적용되는 '출국 후 퇴직금 수령제도'로 인해 이주노동자들이 퇴직금을 받는데 어려움을 겪고 있다. 불법체류를 막기 위해 이주노동자가 출국한 뒤 퇴직금을 받도록 한 '출국만기보험 제도'가 퇴직금 미지급자를 양산하고 있는 것이다. 출국만기보험이란 사업주가 퇴직금을 지급하기 위해 노동자를 피보험자로 해 가입하는 보험이다. 사업주가 매달 통상임금의 8.3%를 보험회사에 적립한 것으로, 노동자가 퇴직 후 출국할 때 공항에서 받을 수 있다. 2014 년 정부는 불법체류 방지를 위해 이주노동자 퇴직금을 '퇴사 후 14 일 이내'에서 '출국 후 14 일 이내' 지급하는 것으로 바꿨다. 하지만 상당수 이주노동자들이 퇴직금 받는 방법을 제대로 모르고 있다. 이주노동자가 퇴직금을 받으려면 구비서류가 많은데다 퇴직금을 수령받는 절차도 복잡하다. 최근 이주노동자를 대상으로한 한 설문조사에 따르면, 출국만기보험금 수령절차를 알고 있는 응답자는 절반도 되지 않았다. 이때문에 미지급된 출국만기보험금은 81 억원에 이른다. 이만이 아니다. 보험금을 빼고 남은 잔액인 잔여퇴직금을 회사에 청구해 받아야 하는데 이를 모르는 이가 50%가 넘었다. 퇴직금 제도가 까다로운데도 이와 관련한 홍보나 교육이 부족했기 때문이다. 이주노동자 관련 단체에서는 정부가 법을 개정한 취지가 불법체류자를 줄이기 위한 것이었으나 불법체류자 감소에 큰 효과를 내지 못하는 만큼 출국 후 퇴직금 수령제도를 철폐해야 한다고 주장하고 있다. 근로기준법은 사업주가 '퇴직' 후 14 일 이내에 퇴직금을 지불해야 한다고 규정하고 있다. 그런데도

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


이주노동자들에게 적용되는 '출국 후 퇴직금 수령제도'로 인해 이주노동자들이 퇴직금을 받는데 어려움을 겪고 있는 가운데 이주노동자들을 대상으로 한 설문조사에 따르면, 출국만기보험금 수령절차를 알고 있는 응답자는 절반도 되지 않는 것으로 나타나 하루빨리 해결책을 마련해 열악한 작업장에서 일하는 이주노동자가 더이상 피해를 보지 않도록 해야 한다.
--------------------------------------------------
예시 9:
원문:
법무부가 제정한 훈령 '형사사건 공개 금지 등에 관한 규정'을 둘러싼 논란이 뜨겁다. 당장 국민의 알권리와 정면 충돌하고 있다는 지적이 비등하고 있다. 법무부의 훈령은 사건 관계인의 인권보호를 위해 검찰의 수사 상황을 전면 비공개하고, 오보 언론에 대한 처벌 수단을 마련하겠다는 게 그 요지다. 구체적으로는 검사와 수사관 등 관계자가 기자와 개별적으로 만날 수 없도록 했다. 또 검찰수사와 관련해 오보를 내면 해당 언론사는 브리핑은 물론 검찰청 출입 자체가 제한된다. 다만 공보 담당자와는 만날 수 있다. 한마디로 기자는 알려주는 내용만 받아쓰라는 것이다. 언론 자유의 침해가 심각하게 우려되는 조치가 아닐 수 없다. 이 규정을 만든 것은 지나친 피의사실 공개로 피의자 인권이 침해당하는 폐해를 막기 위한 조치라는 게 법무부 설명이다. 전혀 틀린 얘기는 아니다. 실제 그동안 검찰이 피의사실을 흘려 망신을 주고, 사실상 여론재판이 이뤄지는 바람에 공정한 재판을 받을 피의자의 권리가 심각하게 훼손되는 경우가 다반사였다. 노무현 전 대통령에 대한 '논두렁 시계' 사건 등이 대표적 사례다. 특히 삼성 등 기업 관련 수사는 미확인 내용이 사실처럼 무차별 유포되면서 기업의 국제신인도가 추락하는 부작용을 낳기도 했다. 이런 폐단을 검찰 개혁 차원에서 없자는 취지는 백번 공감이 간다. 하지만 그 제한이 과도하다는 게 문제다. 당초 취지와도 맞지 않는다. 무엇보다 기자의 브리핑 참석과 출입 제한은 받아들이기 어려운 사안이다. 정보 접근이 어

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


법무부가 제정한 '형사사건 공개 금지 등에 관한 규정'이 국민의 알권리와 정면충돌하고 있다는 지적이 나오고 있는 가운데 오보의 정도가 지나쳐 해당 언론사를 제재할 일이 있더라도 이는 언론 종사사들 간에 자율적으로 정하면 된다.
--------------------------------------------------
예시 10:
원문:
딱 한달이다. 두달도 이어가지 못했다. 5 개월만에 생산·소비·투자 등 경제 활동 3 대 지표가 모두 상승하며 희망을 줬던 산업활동동향(8 월)이 다시 한숨 속 위기감으로 빠져드는데는 한달이면 충분했다. 통계청이 31 일 발표한 '9 월 산업활동 동향'에서 지난달 전(全)산업 생산지수(계절조정계열)는 108.0 으로, 전월보다 0.4% 감소했다. 광공업 생산은 증가했지만, 도소매와 금융·보험업을 중심으로 서비스업 생산이 줄어든 것이 영향을 미쳤다. 소매판매는 음식료품과 의복 판매가 감소하면서 전월보다 2.2%줄어들었다. 감소폭은 2017 년 12 월(-2.4%) 이후 가장 컸다. 이제는 익숙해져 그러려니 하지만 통계청의 설명은 여지없이 기저효과와 날씨 때문이란 설명으로 일관한다. 올해 추석이 이른 9 월이어서 소비가 전달인 8 월으로 분산됐고 9 월엔 잦은 태풍과 장마 등으로 야외활동이 줄어들면서 숙박·음식업과 운수·창고업, 예술·스포츠·여가업 등 서비스업 생산이 감소했다는 것이다. 그 와중에 9 월이 예년비해 기상이 고온이어서 환절기 의류 소비도 감소했다는 설명도 한다. 통신에서도 8 월엔 핸드폰 신제품 출시가 많았고 10 월에 주요 기종 출시 예정된 대기 수요로 9 월 소비는 감소했다고 봤다. 심지어 유일하게 증가한 설비투자도 마찬가지다. 이번엔 역기저효과다. 2~3 년전 반도체장비투자가 워낙 많이 이뤄졌는데 지금은 그 기저효과가 사라져서 소폭이지만 증가하고 있다고 설명한다. 통계청이 경제정책을 입안하고 집행하는 곳은 아니다. 통계를 기초로 정책대안을 제시하라는 것도 아니다. 그건 해당 부처의 몫이다. 물론 통계청은 지난 8

**[대체 모델2] mT5-small**

In [11]:
from transformers import pipeline

# mT5-small용 summarization pipeline 생성
mt5_summarizer = pipeline(
    "summarization",
    model=mt5_model,          # 학습된 mT5-small 모델
    tokenizer=mt5_tokenizer,  # 해당 토크나이저
    device=0                  # GPU 사용 시, CPU면 -1
)

# 검증 데이터셋의 첫 10개 예시에 대해 요약 결과 출력
for i in range(10):
    sample_text = mt5_tokenized_datasets["validation"][i]["text"]
    prompt = f"원문:\n{sample_text}\n요약:"
    print(f"예시 {i+1}:")
    print(prompt)

    # 요약 결과 생성 (필요한 max/min length는 상황에 맞게 조정)
    result = mt5_summarizer(
        prompt,
        max_length=130,
        min_length=30,
        do_sample=False
    )
    print(result[0]["summary_text"])
    print("-" * 50)

Device set to use cuda:0


예시 1:
원문:
더불어민주당 이해찬 대표가 30 일 오후 국회에서 기자간담회를 열고 조국 전 법무부 장관 사태와 관련해 "국민 여러분께 매우 송구하다"고 밝혔다. 더불어민주당 이해찬 대표가 30 일 기자간담회를 열고 '조국 사태'와 관련, "국민 여러분께 매우 송구하다"는 입장을 밝혔다. 이 대표는 "검찰 개혁이란 대의에 집중하다 보니, 국민 특히 청년이 느꼈을 불공정에 대한 상대적 박탈감, 좌절감을 깊이 있게 헤아리지 못했다"며 "여당 대표로서 무거운 책임감을 느낀다"고 머리를 숙였다. 조국 전 법무부 장관이 14 일 사퇴한 이후 이 대표가 당 안팎의 쇄신 요구에 대해 입장을 표명한 것은 이번이 처음이다. 청와대와 여당은 '조국 정국'을 거치며 분출된 '공정'과 '정의'의 민심을 받들어 검찰 개혁에 매진하겠다면서도 두 달간 극심한 분열과 갈등을 초래한데 대해선 진지하게 성찰하는 모습을 보이지 않았다. 그나마 초선인 이철희 의원이 "당이 대통령 뒤에 비겁하게 숨어 있었다"고 비판했고, 표창원 의원은 "책임을 느끼는 분들이 각자 형태로 그 책임감을 행동으로 옮겨야 할 때"라고 지적했다. 뒤늦게나마 이 대표가 자성의 목소리를 내긴 했으나 당 안팎의 쇄신 요구에 어떻게 응할지 구체적 플랜을 제시하지 못해 여전히 안이하다는 지적도 나온다. 이 대표는 28 일 윤호중 사무총장을 단장으로 하는 총선기획단을 발족했고 조만간 인재영입위원회도 출범시킬 계획이라고 밝혔다. 이 대표는 "민주당의 가치를 공유하는 참신한 인물을 영입해 준비된 정책과 인물로 승부하겠다"고 다짐했다. 하지만 당 일각에선 "총선기획단장을 비롯한 당직 인선부터 쇄신 의지를 보여야 한다"는 비판의 목소리가 나온다. 무조건 물러나는 게 능사는 아니지만 국정 혼선을 초래한 데 대해 당 지도부가 겸허하게 책임지는 모습을 보이는 게 쇄신의 출발점이 돼야 한다는 지적도 있다. 선거는 대중의 이해와 요구를 잘 대표하는 정치인을 뽑는 행위다. 민생을 외면하며 낡은 이념과 진영 싸움에 매몰된 구시대 인물들을 과감히 물갈

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


민주당 이해찬 대표가 30 일 기자간담회를 열고 '조국 사태'와 관련, "국민 여러분께 매우 송구하다"고 밝혔는데, 이번이 대중의 대대적인 인적 쇄신에 대해민주당 대표의 책임감을 보여야 할 것이다.
--------------------------------------------------
예시 2:
원문:
탈원전 정책에서 비롯된 한국전력의 경영 부담이 결국 전기료 인상을 초래하게 됐다. 김종갑 한전 사장은 언론 인터뷰에서 "정부 정책에 따라 도입된 각종 전기료 특례할인을 모두 폐지하고 전기요금 원가공개 방안을 정부와 협의하겠다"고 밝혔다. 특례할인은 여름철 누진제, 주택용 절전 및 전기차 충전 등의 명목으로 전기료를 깎아주는 제도로서, 작년 1 조 1434 억원에 달했다. 이 제도가 없어지면 부담은 고스란히 국민에게 돌아가게 된다. 전기료 인상은 예견됐던 것이나 마찬가지다. 문재인 정부 출범 이후 탈원전 정책이 가속화되면서 한전 경영이 악화일로를 달려왔기 때문이다. 원전 가동을 줄이고 발전 단가가 높은 민간 발전사의 전기 구매를 늘린 탓에 한전의 발전 원가는 2016 년 25 조원 수준에서 2018 년 38 조원 수준으로 급등했다. 영업이익이 같은 기간 12 조 16 억원 흑자에서 2080 억원 적자로 급반전한 것은 필연적인 결과다. 적자는 올해 상반기 9285 억원으로 더 불어났다. 우량회사가 2 년여 만에 적자의 수렁에 처박힌 꼴이다. 한전의 추락은 국제신용평가사들의 평가에도 그대로 적용된다. S&P는 그제 한전의 신용등급을 기존 BBB에서 BBB-로 내렸다. 탈원전 정책의 충격이 커지면서 재무 부담이 가중되고 있다는 게 하향 조정 이유다. 표면적으로는 '재무 부담'이란 용어를 사용했지만 한전의 기초체력이 급속히 악화되고 있다는 경고와 다를 바 없다. S&P가 한전의 주요 재무지표에 대해 앞으로 1~2 년간 개선되기 어렵다고 밝힌 점으로 미뤄 신용등급의 추가 하향 조정 가능성도 배제하기 어렵다. 이런 상황에서도 정부는 다음달부터 연말까지 전력효율이 높은 가

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

them은 지난 28 일에 오거돈 부산경남 정치권이 대구경북 통합신공항이전에 트집을 잡고 나섰다.
--------------------------------------------------
예시 8:
원문:
이주노동자들에게 적용되는 '출국 후 퇴직금 수령제도'로 인해 이주노동자들이 퇴직금을 받는데 어려움을 겪고 있다. 불법체류를 막기 위해 이주노동자가 출국한 뒤 퇴직금을 받도록 한 '출국만기보험 제도'가 퇴직금 미지급자를 양산하고 있는 것이다. 출국만기보험이란 사업주가 퇴직금을 지급하기 위해 노동자를 피보험자로 해 가입하는 보험이다. 사업주가 매달 통상임금의 8.3%를 보험회사에 적립한 것으로, 노동자가 퇴직 후 출국할 때 공항에서 받을 수 있다. 2014 년 정부는 불법체류 방지를 위해 이주노동자 퇴직금을 '퇴사 후 14 일 이내'에서 '출국 후 14 일 이내' 지급하는 것으로 바꿨다. 하지만 상당수 이주노동자들이 퇴직금 받는 방법을 제대로 모르고 있다. 이주노동자가 퇴직금을 받으려면 구비서류가 많은데다 퇴직금을 수령받는 절차도 복잡하다. 최근 이주노동자를 대상으로한 한 설문조사에 따르면, 출국만기보험금 수령절차를 알고 있는 응답자는 절반도 되지 않았다. 이때문에 미지급된 출국만기보험금은 81 억원에 이른다. 이만이 아니다. 보험금을 빼고 남은 잔액인 잔여퇴직금을 회사에 청구해 받아야 하는데 이를 모르는 이가 50%가 넘었다. 퇴직금 제도가 까다로운데도 이와 관련한 홍보나 교육이 부족했기 때문이다. 이주노동자 관련 단체에서는 정부가 법을 개정한 취지가 불법체류자를 줄이기 위한 것이었으나 불법체류자 감소에 큰 효과를 내지 못하는 만큼 출국 후 퇴직금 수령제도를 철폐해야 한다고 주장하고 있다. 근로기준법은 사업주가 '퇴직' 후 14 일 이내에 퇴직금을 지불해야 한다고 규정하고 있다. 그런데도 이주노동자에게만 그 시점을 '출국'으로 다르게 적용하는 것은 기본권 침해라는 지적도 나오고 있다. 소멸시효 3 년이 지날 때까지 이주노동자가 찾아가지 않은 미지급금이 많다보

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<extra_id_0>자들에게 적용되는 '출국 후 퇴직금' 수령제도'를 적용하는 '출국 후 퇴직금 수령제도'를 철제해야 한다.
--------------------------------------------------
예시 9:
원문:
법무부가 제정한 훈령 '형사사건 공개 금지 등에 관한 규정'을 둘러싼 논란이 뜨겁다. 당장 국민의 알권리와 정면 충돌하고 있다는 지적이 비등하고 있다. 법무부의 훈령은 사건 관계인의 인권보호를 위해 검찰의 수사 상황을 전면 비공개하고, 오보 언론에 대한 처벌 수단을 마련하겠다는 게 그 요지다. 구체적으로는 검사와 수사관 등 관계자가 기자와 개별적으로 만날 수 없도록 했다. 또 검찰수사와 관련해 오보를 내면 해당 언론사는 브리핑은 물론 검찰청 출입 자체가 제한된다. 다만 공보 담당자와는 만날 수 있다. 한마디로 기자는 알려주는 내용만 받아쓰라는 것이다. 언론 자유의 침해가 심각하게 우려되는 조치가 아닐 수 없다. 이 규정을 만든 것은 지나친 피의사실 공개로 피의자 인권이 침해당하는 폐해를 막기 위한 조치라는 게 법무부 설명이다. 전혀 틀린 얘기는 아니다. 실제 그동안 검찰이 피의사실을 흘려 망신을 주고, 사실상 여론재판이 이뤄지는 바람에 공정한 재판을 받을 피의자의 권리가 심각하게 훼손되는 경우가 다반사였다. 노무현 전 대통령에 대한 '논두렁 시계' 사건 등이 대표적 사례다. 특히 삼성 등 기업 관련 수사는 미확인 내용이 사실처럼 무차별 유포되면서 기업의 국제신인도가 추락하는 부작용을 낳기도 했다. 이런 폐단을 검찰 개혁 차원에서 없자는 취지는 백번 공감이 간다. 하지만 그 제한이 과도하다는 게 문제다. 당초 취지와도 맞지 않는다. 무엇보다 기자의 브리핑 참석과 출입 제한은 받아들이기 어려운 사안이다. 정보 접근이 어렵거나 취재원의 한정과 시간상 제약 등으로 의도치 않은 '오보'는 보도는 과정에서 필연적일 수밖에 없다. 단지 이를 이유로 출입을 차단하고 받아쓰기만 강요한다면 더 이상 언론은 그 기능을 유지하기 어렵다. 정부가 특정

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


를 위해 검찰의 수사 상황을 전면 비공개하고, 오보를 내면 해당 언론에 재판을 받아쓰야 한다.
--------------------------------------------------
예시 10:
원문:
딱 한달이다. 두달도 이어가지 못했다. 5 개월만에 생산·소비·투자 등 경제 활동 3 대 지표가 모두 상승하며 희망을 줬던 산업활동동향(8 월)이 다시 한숨 속 위기감으로 빠져드는데는 한달이면 충분했다. 통계청이 31 일 발표한 '9 월 산업활동 동향'에서 지난달 전(全)산업 생산지수(계절조정계열)는 108.0 으로, 전월보다 0.4% 감소했다. 광공업 생산은 증가했지만, 도소매와 금융·보험업을 중심으로 서비스업 생산이 줄어든 것이 영향을 미쳤다. 소매판매는 음식료품과 의복 판매가 감소하면서 전월보다 2.2%줄어들었다. 감소폭은 2017 년 12 월(-2.4%) 이후 가장 컸다. 이제는 익숙해져 그러려니 하지만 통계청의 설명은 여지없이 기저효과와 날씨 때문이란 설명으로 일관한다. 올해 추석이 이른 9 월이어서 소비가 전달인 8 월으로 분산됐고 9 월엔 잦은 태풍과 장마 등으로 야외활동이 줄어들면서 숙박·음식업과 운수·창고업, 예술·스포츠·여가업 등 서비스업 생산이 감소했다는 것이다. 그 와중에 9 월이 예년비해 기상이 고온이어서 환절기 의류 소비도 감소했다는 설명도 한다. 통신에서도 8 월엔 핸드폰 신제품 출시가 많았고 10 월에 주요 기종 출시 예정된 대기 수요로 9 월 소비는 감소했다고 봤다. 심지어 유일하게 증가한 설비투자도 마찬가지다. 이번엔 역기저효과다. 2~3 년전 반도체장비투자가 워낙 많이 이뤄졌는데 지금은 그 기저효과가 사라져서 소폭이지만 증가하고 있다고 설명한다. 통계청이 경제정책을 입안하고 집행하는 곳은 아니다. 통계를 기초로 정책대안을 제시하라는 것도 아니다. 그건 해당 부처의 몫이다. 물론 통계청은 지난 8 월 경제 지표에도 호들갑을 떨며 큰 의미를 부여하지는 않았다. 수출이나 대외 여건이 개선되지 않은 상황에서 확실한 상승세로 보기는 

- 세 모델 모두 동일한 검증 샘플 10개에 대해 요약을 생성했다.

- `KoBART`와 `KoT5`는 원문의 핵심 문장과 논조를 비교적 잘 따라가며, 기사 전체의 주제 문장을 중심으로 요약을 구성하는 경향을 보였다.
​
- `mT5-small`은 짧은 문장, 문법 오류, `<extra_id_0`> 같은 토큰 노출 등으로 인해 의미가 끊기거나 원문 논지를 충분히 정리하지 못하는 경우가 자주 나타났다.

### 4.2. [정성 평가] 원문–정답–모델 요약 사례 비교

검증셋 10개에 대해 “원문–정답 요약–모델 요약”을 비교한 결과, 다음과 같은 패턴이 보였다.

- **KoBART**
    - 장점: 사설의 핵심 주장(예: 이해찬 대표 사과 후 인적 쇄신 필요, 탈원전–전기료 인상, 디플레 우려 등)을 비교적 풍부하게 담고, 문장 수도 길어 정보량이 많다.
    - 약점: 동일 문구 반복(“인적 쇄신으로 이어지길 기대한다” 등)과 약간의 장황함이 나타나, “요약이라기보다 논설 재서술”에 가까운 느낌을 줄 때가 있다.

- **KoT5**
    - 장점: KoBART보다 한두 문장 더 압축된 형태로, 주제 문장을 비교적 자연스럽고 간결하게 뽑아낸다.
    - 약점: 일부 샘플에서 세부 근거 문장을 생략하여 “왜 그런 결론에 이르렀는지”가 다소 약하게 표현되는 경우가 있다.

- **mT5-small**
    - 장점: 특정 키워드는 잡지만, 전체 구조를 충실히 재구성하기보다는 고립된 문장, 잘린 구문, 의미 불명 문장(“them은 …”, “오는 한숨 속 속 위기감으로…” 등)이 자주 출현한다.
    - 약점: 정답 요약과 비교했을 때 논리적 흐름과 핵심 주장 연결이 많이 깨져, 실제 활용에는 적합하지 않은 수준의 품질을 보인다.

- **정성 평가**
    - KoBART: 핵심 주장과 근거 일부를 충분히 담는 요약이 필요할 때 적합함.
    - KoT5: 조금 더 압축된 한두 문장 요약이 필요할 때 상대적으로 적합함.
    - mT5-small: 파라미터 규모 대비 실험용·비교용으로 의미가 있지만 실제 서비스 품질에는 미치지 못함.

### 4.3. [정량 평가] ROUGE 계산 및 모델별 성능 비교 표

#### 공통 평가 함수 설계
- 입력:
    - `model`, `tokenizer`,
    - `dataset` (예: validation 일부),
    - 평가에 사용할 샘플 개수 `n_samples`

- 처리:
    - 앞에서부터 n_samples개 원문을 가져와 model.generate 또는 지금처럼 pipeline으로 요약 생성
    - 정답 요약과 함께 ROUGE 계산
- 출력:
    - ROUGE-1/2/L의 평균 점수 딕셔너리

In [10]:
from rouge_score import rouge_scorer

def evaluate_rouge_with_pipeline(summarizer, dataset, n_samples=50):
    # ROUGE-L까지 포함한 scorer 준비
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=False)
    scores = {"rouge1": [], "rouge2": [], "rougeL": []}

    for i in range(n_samples):
        ref = dataset[i]["summary"]
        src = dataset[i]["text"]

        # 필요하면 src를 자르거나 프롬프트 붙이기
        result = summarizer(src, max_new_tokens=128, do_sample=False)
        pred = result[0]["summary_text"]

        score = scorer.score(ref, pred)
        for k in scores:
            scores[k].append(score[k].fmeasure)

    avg_scores = {k: sum(v) / len(v) for k, v in scores.items()}
    return avg_scores

**[베이스모델] KoBART**

In [11]:
# KoBART용 summarizer는 이미 학습 후에 만들어 둔 것 사용
kobart_rouge = evaluate_rouge_with_pipeline(
    summarizer=summarizer,  # KoBART용 pipeline
    dataset=tokenized_datasets["validation"],  # 또는 원본 validation 리스트
    n_samples=50,
)
print(kobart_rouge)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{'rouge1': 0.22586080586080587, 'rouge2': 0.06587878787878788, 'rougeL': 0.21919413919413916}


**[대체 모델1] KoT5 한국어 요약 모델**

In [14]:
from transformers import pipeline

# KoT5 summarization pipeline (평가용)
kot5_summarizer = pipeline(
    "summarization",
    model=kot5_model,
    tokenizer=kot5_tokenizer,
    device=0
)

# KoT5 정량 평가 실행 (KoBART / mT5와 같은 함수 재사용)
kot5_rouge = evaluate_rouge_with_pipeline(
    summarizer=kot5_summarizer,
    dataset=valid_examples,   # {"text", "summary"} 리스트
    n_samples=50
)

print("KoT5 ROUGE:", kot5_rouge)

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


KoT5 ROUGE: {'rouge1': 0.2243333333333333, 'rouge2': 0.08666666666666666, 'rougeL': 0.2243333333333333}


**[대체 모델2] mT5-small**

In [14]:
from transformers import pipeline
from rouge_score import rouge_scorer

# 1) mT5-small 모델과 토크나이저 로드 (이미 학습된 상태라고 가정)
mt5_model_name = "google/mt5-small"
mt5_tokenizer = AutoTokenizer.from_pretrained(mt5_model_name)
mt5_model = AutoModelForSeq2SeqLM.from_pretrained(mt5_model_name)

# 2) summarization pipeline 생성
mt5_summarizer = pipeline(
    "summarization",
    model=mt5_model,
    tokenizer=mt5_tokenizer,
    device=0 
)

# 3) mT5-small 정량 평가 실행
mt5_rouge = evaluate_rouge_with_pipeline(
    summarizer=mt5_summarizer,
    dataset= valid_examples,
    n_samples=50
)
print("mT5-small ROUGE:", mt5_rouge)

Device set to use cuda:0


mT5-small ROUGE: {'rouge1': 0.005, 'rouge2': 0.0, 'rougeL': 0.005}


#### 정량 평가

같은 50개 검증 샘플에 대해 ROUGE-1/2/L F1을 계산한 결과는 다음과 같다.

| 모델        | ROUGE-1 | ROUGE-2 | ROUGE-L |
| --------- | ------- | ------- | ------- |
| KoBART    | 0.2259  | 0.0659  | 0.2192  |
| KoT5      | 0.2243  | 0.0867  | 0.2243  |
| mT5-small | 0.0050  | 0.0000  | 0.0050  |

- ROUGE-1 (단어 수준 정보 보존): KoBART와 KoT5가 거의 유사한 수준으로, 정답 요약과 겹치는 핵심 단어를 비슷한 정도로 포함한다는 의미다.
​
- ROUGE-2 (2-그램, 짧은 표현·구문 보존): KoT5가 KoBART보다 높아, “연속된 표현 단위”를 기준 요약과 더 비슷하게 이어가는 경향이 있다.
​
- ROUGE-L (문장 전체 구조·순서 보존): KoT5가 약간 높고, KoBART도 비슷한 수준으로, 두 모델 모두 전체 문장 구조를 정답과 어느 정도 비슷하게 유지하고 있음을 보여준다.

- mT5-small은 세 지표가 모두 거의 0에 가까워, 참조 요약과의 중복이 매우 적고 구조도 잘 맞지 않음을 정량적으로 확인시켜 준다.

모델별 학습 과정에 따른 loss 는 다음과 같다.

| 모델        | Step 샘플 | Training Loss (예시) |
| --------- | ------- | ------------------ |
| KoBART    | 500     | 2.1962             |
| KoBART    | 1000    | 1.7078             |
| KoBART    | 1500    | 1.4554             |
| KoT5      | 500     | 1.6915             |
| KoT5      | 1000    | 1.6097             |
| KoT5      | 1500    | 1.5429             |
| mT5-small | 500     | 5.1609             |
| mT5-small | 1000    | 3.0701             |
| mT5-small | 1500    | 2.9095             |

- KoBART: 짧은 학습 시간(약 14분)에도 불구하고 안정적인 train loss 감소와 적당한 길이의 자연스러운 문장 생성이 가능했다.

- KoT5: 학습 시간은 가장 길었지만(train 3epoch에 약 8시간대), train loss가 1.6대까지 떨어지면서 ROUGE-2·L에서 KoBART와 비슷하거나 약간 우수한 결과를 보였다.
​
- mT5-small: 학습 시간은 상대적으로 짧았으나(약 3시간), train loss는 높고(3~5대) 정성·정량 지표 모두에서 품질이 떨어져, 파라미터 규모와 사전학습 도메인 한계가 함께 드러난다.

## 5. 전체 워크플로우 정리 및 마무리

### 5.1. [파이프라인 정리] 모델별 학습 및 평가 요약

- 전체 파이프라인은 `데이터 로드` → `전처리` → `모델별 파인튜닝` → `검증 샘플 10개 정성 평가` → `검증 샘플 50개 정량 평가` → `결과 비교` 의 흐름으로 구성되었다.

- 아래 표는 모델별 학습시간, loss 추이, 정성·정량 평가 결과를 한눈에 정리한 것이다.


    | 항목               | KoBART (베이스 모델)                              | KoT5 (대체 모델1)                               | mT5-small (대체 모델2)                          |
    | ---------------- | ----------------------------------------- | ----------------------------------------- | ----------------------------------------- |
    | 학습 시간(3epoch)    | 약 14분대                                    | 약 8시간대                                    | 약 3시간대                                    |
    | Training Loss 예시 | 2.20 → 1.71 → 1.46 (step 500, 1000, 1500) | 1.69 → 1.61 → 1.54 (step 500, 1000, 1500) | 5.16 → 3.07 → 2.91 (step 500, 1000, 1500) |
    | ROUGE-1          | 0.2259                                    | 0.2243                                    | 0.0050                                    |
    | ROUGE-2          | 0.0659                                    | 0.0867                                    | 0.0000                                    |
    | ROUGE-L          | 0.2192                                    | 0.2243                                    | 0.0050                                    |
    | 정성 평가 요약         | 핵심 내용은 잘 잡지만, 유사 문장 반복·장황함 다소 큼           | 간결하고 자연스러운 요약, 일부 사례에서 근거 설명이 부족함         | 짧고 어색한 문장, 구조 붕괴·잘림 현상 빈번                 |
    | 총평               | 정보량 풍부, 후처리(반복 제거) 필요                     | 가독성·자연스러움 측면에서 최종 추천 모델로 적합               | 실험·비교용으로 의미, 실제 서비스에는 부적합                 |
</br>

- 학습 안정성(Training Loss)은 `KoBART`·`KoT5`가 비슷하지만, `KoT5`가 ROUGE-2와 ROUGE-L이 약간 더 좋다.

- 정성/정량 평가 Trade off 관점에서, `KoBART`는 정보량·중복 단어 기준 점수는 괜찮지만 문장 품질이 떨어지고, `KoT5`는 점수는 비슷하면서 실제 읽기에도 더 자연스럽다.

- **종합적으로, 실사용을 가정한 최종 추천 모델은 `KoT5`이다.**

- `KoBART`는 짧은 학습 시간에도 안정적인 성능을 보여주어, 대안으로 활용 가능하다.

- `mT5-small` 의 경우, 소형 모델의 한계와 자원 제약 환경에서의 성능 저하 양상을 확인할 수 있었다.

### 5.2. [실험 인사이트] 모델 비교 결과 정리 및 확장 아이디어 제안

세 모델 비교에서 얻은 핵심 인사이트는 다음과 같다.

- 모델 크기와 품질의 관계
   - 단순히 작은 모델(mT5-small)을 쓰면 메모리에는 유리하지만, 한국어 사설 요약 품질은 ROUGE와 정성 평가 모두에서 크게 떨어진다.

   - 일정 수준 이상의 파라미터와 한국어 특화 사전학습이 있어야, 실용적인 요약 품질이 나온다는 점을 확인했다.

- 정량 지표와 읽기 경험의 차이
   - `KoBART`와 `KoT5`는 ROUGE-1/L이 비슷하고 ROUGE-2에서도 큰 차이는 아니지만, 사람이 읽을 때 느끼는 “좋은 요약”의 인상은 꽤 다르다.

   - 즉, ROUGE만으로는 문장 반복, 부자연스러움, 구조적 어색함을 충분히 포착하기 어렵고, 반드시 정성 평가를 함께 해야 한다는 교훈을 준다.

- 환경 제약이 모델 선택에 미치는 영향
   - `T5-base` 한국어 요약 모델은 OOM으로 학습 불가였고, 이로 인해 `KoT5`·`mT5-small`처럼 “환경에 맞는 중·소형 모델”을 선택할 수밖에 없었다.

   - 리소스 제약을 명시하고, 그 안에서 달성 가능한 최선의 모델 구성을 찾는 과정 자체가 중요한 설계 포인트가 된다.